# This notebook will attempt to show how to compare two models

We often arrive in a situation where we've got multiple different models. Yet we're note sure which one we should focus on or start from for a particular task.
This notebook aims to introduce some tools that (hopefully) help us do that.

## Initial input - models and data

There are two different workflows this notebook can handle:
1. Compare two different model packs
  - Provide 2 model pack paths
  - Provide a documents file
2. Compare model pack with and without supervised training
  - Provide 1 model pack path
  - Provide a file path to a MedCATtrainer (MCT) export
  - Provide a document file

The model packs can be either the `.zip` file (which will be automatically unzipped) or the folder.

The documents file is expected in a `.csv` format with two columns (`id`, and `text`).

The MCT export is expected in the format given by MedCATtrainer.

For the two approaches, there is a slightly different internal workflow.
But other than ticking the checkbox, the process should be identical to the user.

### CUI filter settings

These are optional.

If you wish to filter based on CUIs (i.e only run the comparison for some CUIs), you can do so.
You can either list the CUIs (separated by comma) or provide a file that lists them (separated by comma).

You can also include the children of the selected CUIs.The default is not to do so.
But you can opt to include children of a certain order (i.e `1` means direct children only, `2` means children of children as well, and so on).

In [1]:
from comp_nbhelper import NBInputter
inputs = NBInputter()
inputs.show_all()

/Users/martratas/Documents/CogStack/.MedCAT.nosync/working_with_cogstack/venv310/lib/python3.10/site-packages/medcat/cat.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


### Running the difference finder

Now that we've got the input data, we need to figure out how they work and what their differences are.
We use the `get_comparison` method that loads both models, runs `CAT.get_entities` on each document for either model, and then returns some results.

These results show describe the difference in the raw CDB (i.e the number of concepts (join and unique), amount of training, and so on), the total differences in the entities extracted (i.e the number of recognitions and forms per CUI) as well as per document differences (i.e the number of identical as well as different entity recognitions found).

We will look into the details later.

In [2]:
comparison = inputs.get_comparison()

For models, selected:
Model1: /Users/martratas/Documents/CogStack/.MedCAT.nosync/MedCAT/models/20230227__kch_gstt_trained_model_494c3717f637bb89.zip
Model2: /Users/martratas/Documents/CogStack/.MedCAT.nosync/MedCAT/models/snomed2024-06-mimic-iv.zip
Documents: /Users/martratas/Documents/CogStack/.MedCAT.nosync/working_with_cogstack/medcat/compare_models/data/intechopen_2cols_3.csv
For CUI filter, selected:
Filter: None
Children: None
Loading [1] /Users/martratas/Documents/CogStack/.MedCAT.nosync/MedCAT/models/20230227__kch_gstt_trained_model_494c3717f637bb89.zip


/Users/martratas/Documents/CogStack/.MedCAT.nosync/working_with_cogstack/venv310/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Loading [2] /Users/martratas/Documents/CogStack/.MedCAT.nosync/MedCAT/models/snomed2024-06-mimic-iv.zip


/Users/martratas/Documents/CogStack/.MedCAT.nosync/working_with_cogstack/venv310/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Per annotations diff finding


14370it [6:47:45,  1.70s/it]


Counting [1&2]


100%|██████████| 14362/14362 [02:52<00:00, 83.23it/s] 


CDB compare


keys: 100%|██████████| 804513/804513 [00:03<00:00, 259249.78it/s]


For now, we'll use the common parser/display method to display an overview of the results.
We can later look at more granual details as well.

In [3]:
comparison.show_all()

CDB overall differences:


| Path | Value | [Optional] Comparison |
| ----- | ----- | ----- |
| names.keys.joint                         | 750494                                   |                                          |
| names.keys.not_in_                       | 44230                                    | 9789                                     |
| names.keys.total                         | 760283                                   | 794724                                   |
| names.values.joint                       | 2659786                                  |                                          |
| names.values.unique_in_                  | 421061                                   | 251871                                   |
| names.values.not_in_                     | 375036                                   | 457386                                   |
| names.values.total                       | 3149859                                  | 3067509                                  |
| snames.keys.joint                        | 750494                                   |                                          |
| snames.keys.not_in_                      | 44230                                    | 9789                                     |
| snames.keys.total                        | 760283                                   | 794724                                   |
| snames.values.joint                      | 5697138                                  |                                          |
| snames.values.unique_in_                 | 962832                                   | 552795                                   |
| snames.values.not_in_                    | 1273285                                  | 1291058                                  |
| snames.values.total                      | 13486640                                 | 13468867                                 |

Now tally differences


| Path | First | Second |
| ----- | ----- | ----- |
| pt2ch (Dict[str, Set])                   | 352226 keys (mean 2.0 values per key)    | 372046 keys (mean 2.0 values per key)    |
| cat_data                                 | {'Number of concepts': 760283, 'Number of names': 3080845, 'Number of concepts that received training': 38460, 'Number of seen training examples in total': 153875883, 'Average training examples per concept': 4000.932995319813} | {'Number of concepts': 794724, 'Number of names': 2911657, 'Number of concepts that received training': 62855, 'Number of seen training examples in total': 253418075, 'Average training examples per concept': 4031.788640521836} |
| per_cui_count (Dict[str, int])           | 45670 keys (total 5838291 in value)      | 42846 keys (total 5433652 in value)      |
| per_cui_acc (Dict[str, float])           | 45670 keys (mean 0.9514984470889257 in value) | 42846 keys (mean 0.9727027021541499 in value) |
| per_cui_forms (Dict[str, Set])           | 45670 keys (mean 2.0 values per key)     | 42846 keys (mean 2.0 values per key)     |
| per_type_counts (Dict[str, int])         | 51 keys (total 5838259 in value)         | 51 keys (total 5433863 in value)         |
| total_count                              | 5838291                                  | 5433652                                  |

Now per-annotation differences:


| Path | Value | [Optional] Comparison |
| ----- | ----- | ----- |
| IDENTICAL                                | 4325640                                  |                                          |
| SAME_SPAN_DIFF_CONCEPT                   | 498167                                   |                                          |
| SECOND_HAS                               | 318069                                   |                                          |
| FIRST_HAS                                | 722708                                   |                                          |
| OVERLAPP_2ND_LARGER_DIFF_CONCEPT         | 33939                                    |                                          |
| OVERLAPP_1ST_LARGER_DIFF_CONCEPT         | 48401                                    |                                          |
| SAME_PARENT                              | 124810                                   |                                          |
| SAME_SPAN_CONCEPT_NOT_IN_2ND             | 66826                                    |                                          |
| SAME_GRANDPARENT                         | 7939                                     |                                          |
| SAME_SPAN_CONCEPT_NOT_IN_1ST             | 6256                                     |                                          |
| OVERLAPP_1ST_LARGER_SAME_CONCEPT         | 2125                                     |                                          |
| OVERLAPP_2ND_LARGER_SAME_CONCEPT         | 1386                                     |                                          |
| PARTIAL_OVERLAP_DIFF_CONCEPT             | 72                                       |                                          |
| PARTIAL_OVERLAP_SAME_CONCEPT             | 22                                       |                                          |

## More granual details (per document view)

The above does not give us all the information we need.
For instance, we may also want to compare the performance accross some documents.
We can do so as follows.

In [4]:
# you can play with individual parts as well.
# for example, isolate a specific document
comparison.show_per_document(limit=10)

Chemokines Updates_1. Chemokines Effective on Platelet Functions 


| Path | Value | [Optional] Comparison |
| ----- | ----- | ----- |
| IDENTICAL                                | 416                                      |                                          |
| SAME_SPAN_DIFF_CONCEPT                   | 38                                       |                                          |
| SECOND_HAS                               | 21                                       |                                          |
| FIRST_HAS                                | 38                                       |                                          |
| OVERLAPP_2ND_LARGER_DIFF_CONCEPT         | 4                                        |                                          |
| OVERLAPP_1ST_LARGER_DIFF_CONCEPT         | 5                                        |                                          |
| SAME_PARENT                              | 12                                       |                                          |
| SAME_SPAN_CONCEPT_NOT_IN_2ND             | 2                                        |                                          |
| SAME_GRANDPARENT                         | 2                                        |                                          |

Chemokines Updates_2. Alteration of Cytokines Level and Oxidative Stress Parameters in COVID-19 


| Path | Value | [Optional] Comparison |
| ----- | ----- | ----- |
| IDENTICAL                                | 194                                      |                                          |
| SAME_SPAN_DIFF_CONCEPT                   | 27                                       |                                          |
| FIRST_HAS                                | 27                                       |                                          |
| SAME_PARENT                              | 3                                        |                                          |
| OVERLAPP_1ST_LARGER_DIFF_CONCEPT         | 3                                        |                                          |
| SECOND_HAS                               | 10                                       |                                          |
| SAME_SPAN_CONCEPT_NOT_IN_2ND             | 3                                        |                                          |
| SAME_SPAN_CONCEPT_NOT_IN_1ST             | 2                                        |                                          |

Chemokines Updates_3. Chemokines at the Crossroad of Diabetes-Tuberculosis Synergy 


...

| Path | Value | [Optional] Comparison |
| ----- | ----- | ----- |
| IDENTICAL                                | 152                                      |                                          |
| SAME_SPAN_DIFF_CONCEPT                   | 32                                       |                                          |
| FIRST_HAS                                | 19                                       |                                          |
| SECOND_HAS                               | 17                                       |                                          |
| SAME_PARENT                              | 7                                        |                                          |
| OVERLAPP_1ST_LARGER_DIFF_CONCEPT         | 1                                        |                                          |

Cerebral Palsy_2. Management of Spasticity and Cerebral Palsy Update 


| Path | Value | [Optional] Comparison |
| ----- | ----- | ----- |
| IDENTICAL                                | 183                                      |                                          |
| SAME_SPAN_DIFF_CONCEPT                   | 14                                       |                                          |
| FIRST_HAS                                | 16                                       |                                          |
| SECOND_HAS                               | 16                                       |                                          |
| OVERLAPP_2ND_LARGER_DIFF_CONCEPT         | 1                                        |                                          |
| OVERLAPP_1ST_LARGER_DIFF_CONCEPT         | 5                                        |                                          |
| SAME_PARENT                              | 2                                        |                                          |

Cerebral Palsy_3. Anatomical Surface Guided Techniques for Botulinum Toxin Injection in Spastic Cerebral Palsy Children 


| Path | Value | [Optional] Comparison |
| ----- | ----- | ----- |
| SECOND_HAS                               | 24                                       |                                          |
| IDENTICAL                                | 136                                      |                                          |
| OVERLAPP_1ST_LARGER_DIFF_CONCEPT         | 1                                        |                                          |
| OVERLAPP_2ND_LARGER_DIFF_CONCEPT         | 4                                        |                                          |
| SAME_SPAN_DIFF_CONCEPT                   | 23                                       |                                          |
| FIRST_HAS                                | 18                                       |                                          |
| SAME_SPAN_CONCEPT_NOT_IN_2ND             | 3                                        |                                          |
| SAME_PARENT                              | 2                                        |                                          |

# Saving annotation output to CSV file
You can also save the annotation output to a .csv file. That file inclues the following columns:
```
doc_id  text    ann1    ann2
```
where `doc_id` refers to the ID of the document in question, `text` is the relevant text around the specific annotation, `ann1` is the annotation json for model 1 (if present), and `ann2` is the annotation json for model 2 (if present).

*Note:* One of the annotations may not be present. This is the case if one of the models did not annotate that specific span.

In [ ]:
comparison.diffs_to_csv("23vs24_annotations.csv")

## More granual details (per cui view)

We may also want to look at how we did for a specific CUI.
This is how we can do that.

In [5]:
# cui = '37151006'  # Erythromelalgia
cui = '25064002'  # headache
comparison.compare_for_cui(cui)

| Path | First | Second |
| ----- | ----- | ----- |
| name                                     | Headache (and 76 children)               | Headache (and 96 children)               |
| count                                    | 1150                                     | 1263                                     |
| acc                                      | 27.412337008966013                       | 29.86313578127611                        |
| forms                                    | 50                                       | 52                                       |

## More granual details (per annotation view)
Sometimes we may want to look at things on a per annotation basis as well.
That is, we want to look at some annotations and compare them between the two models.

In [11]:
# we can iterate over annotation pairs.
# we may optionally specify the documents we wish to look at
# we will specify one document here so as to not generate too much output
docs = ['Chemokines Updates_1. Chemokines Effective on Platelet Functions']
# by default, this will omit identical annotations
# but this can be changed by setting omit_identical=False
comparison.show_docs(docs, omit_identical=True)

Chemokines Updates_1. Chemokines Effective on Platelet Functions (AnnotationComparisonType.SAME_SPAN_DIFF_CONCEPT) 


| Path | First | Second |
| ----- | ----- | ----- |
| pretty_name                              | Chemical                                 | Chemical                                 |
| cui                                      | 723701000000100                          | 441900009                                |
| type_ids                                 | ['7882689']                              | ['91187746']                             |
| types                                    | ['']                                     | ['substance']                            |
| source_value                             | chemical                                 | chemical                                 |
| detected_name                            | chemical                                 | chemical                                 |
| acc                                      | 0.99                                     | 0.673498179708426                        |
| context_similarity                       | 0.99                                     | 0.673498179708426                        |
| start                                    | 49                                       | 49                                       |
| end                                      | 57                                       | 57                                       |
| icd10                                    | []                                       | []                                       |
| ontologies                               | ['20220803_SNOMED_UK_CLINICAL_EXT']      | ['SNOMED-CT']                            |
| snomed                                   | []                                       | []                                       |
| id                                       | 1                                        | 1                                        |
| meta_anns (Dict[str, dict])              | 3                                        | 0                                        |

Chemokines Updates_1. Chemokines Effective on Platelet Functions (AnnotationComparisonType.SAME_SPAN_DIFF_CONCEPT) 


| Path | First | Second |
| ----- | ----- | ----- |
| pretty_name                              | Cell count                               | Cell structure                           |
| cui                                      | 118218001                                | 4421005                                  |
| type_ids                                 | ['28321150']                             | ['66527446']                             |
| types                                    | ['']                                     | ['cell structure']                       |
| source_value                             | cell                                     | cell                                     |
| detected_name                            | cell                                     | cell                                     |
| acc                                      | 0.6106036173080597                       | 0.99                                     |
| context_similarity                       | 0.6106036173080597                       | 0.99                                     |
| start                                    | 225                                      | 225                                      |
| end                                      | 229                                      | 229                                      |
| icd10                                    | []                                       | []                                       |
| ontologies                               | ['20220803_SNOMED_UK_CLINICAL_EXT']      | ['SNOMED-CT']                            |
| snomed                                   | []                                       | []                                       |
| id                                       | 11                                       | 11                                       |
| meta_anns (Dict[str, dict])              | 3                                        | 0                                        |

Chemokines Updates_1. Chemokines Effective on Platelet Functions (AnnotationComparisonType.SECOND_HAS) 


...

| Path | First | Second |
| ----- | ----- | ----- |
| pretty_name                              | Cell count                               | Cell structure                           |
| cui                                      | 118218001                                | 4421005                                  |
| type_ids                                 | ['28321150']                             | ['66527446']                             |
| types                                    | ['']                                     | ['cell structure']                       |
| source_value                             | cell                                     | cell                                     |
| detected_name                            | cell                                     | cell                                     |
| acc                                      | 0.7891191722840991                       | 0.99                                     |
| context_similarity                       | 0.7891191722840991                       | 0.99                                     |
| start                                    | 10834                                    | 10834                                    |
| end                                      | 10838                                    | 10838                                    |
| icd10                                    | []                                       | []                                       |
| ontologies                               | ['20220803_SNOMED_UK_CLINICAL_EXT']      | ['SNOMED-CT']                            |
| snomed                                   | []                                       | []                                       |
| id                                       | 671                                      | 643                                      |
| meta_anns (Dict[str, dict])              | 3                                        | 0                                        |

Chemokines Updates_1. Chemokines Effective on Platelet Functions (AnnotationComparisonType.OVERLAPP_1ST_LARGER_DIFF_CONCEPT) 


| Path | First | Second |
| ----- | ----- | ----- |
| pretty_name                              | Tumor necrosis factor                    | Tumor necrosis                           |
| cui                                      | 8612007                                  | 34823008                                 |
| type_ids                                 | ['91187746']                             | ['33782986']                             |
| types                                    | ['']                                     | ['morphologic abnormality']              |
| source_value                             | Tumor necrosis factor-alpha              | Tumor necrosis                           |
| detected_name                            | tumor~necrosis~factor~alpha              | tumor~necrosis                           |
| acc                                      | 0.7116768890085855                       | 1.0                                      |
| context_similarity                       | 0.7116768890085855                       | 1.0                                      |
| start                                    | 10901                                    | 10901                                    |
| end                                      | 10928                                    | 10915                                    |
| icd10                                    | []                                       | []                                       |
| ontologies                               | ['20220803_SNOMED_UK_CLINICAL_EXT']      | ['SNOMED-CT']                            |
| snomed                                   | []                                       | []                                       |
| id                                       | 675                                      | 645                                      |
| meta_anns (Dict[str, dict])              | 3                                        | 0                                        |

Chemokines Updates_1. Chemokines Effective on Platelet Functions (AnnotationComparisonType.SECOND_HAS) 


| Path | First | Second |
| ----- | ----- | ----- |
| pretty_name                              |                                          | Greek letter alpha                       |
| cui                                      |                                          | 263660008                                |
| type_ids                                 |                                          | ['7882689']                              |
| types                                    |                                          | ['qualifier value']                      |
| source_value                             |                                          | alpha                                    |
| detected_name                            |                                          | alpha                                    |
| acc                                      |                                          | 1.0                                      |
| context_similarity                       |                                          | 1.0                                      |
| start                                    |                                          | 10923                                    |
| end                                      |                                          | 10928                                    |
| icd10                                    |                                          | []                                       |
| ontologies                               |                                          | ['SNOMED-CT']                            |
| snomed                                   |                                          | []                                       |
| id                                       |                                          | 649                                      |
| meta_anns (Dict[str, dict])              | 0                                        | 0                                        |